In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# !pip install rich

In [ ]:
import sys
sys.path.append("../src/agents")

from rich.console import Console
console = Console()

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
# Fix import error by explicitly importing ChatHistoryAgentThread
from semantic_kernel.contents import ChatHistory, ChatMessageContent
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Update the order_team.py file to use the modified agents
from order.order_team import processing_team

In [ ]:
history = ChatHistory()

# Read order from file in /data
with open("data/input_order_1.json", "r") as file:
    order = file.read()

history.add_user_message(f"Please process the following order with detailed markdown table summaries after each step: {order}")

async for response in processing_team.invoke(history=history):
    msg: ChatMessageContent = response
    logger.info(f"{msg.name}: {msg.content}")

In [ ]:
# Extract and display all markdown tables from the conversation history
import re

def extract_markdown_tables(text):
    # Regex to find markdown tables with headers and at least one row
    pattern = r"(\|[^\|]+\|[^\|]+\|.*\n\|\s*[-:]+\s*\|\s*[-:]+\s*\|.*\n(\|.*\|.*\n)+)"
    tables = re.findall(pattern, text)
    return [table[0] for table in tables]

for i, msg in enumerate(history.messages):
    if msg.content and msg.role == 'assistant':
        tables = extract_markdown_tables(msg.content)
        if tables:
            print(f"\n--- {msg.name} ORDER SUMMARY TABLE ---")
            for table in tables:
                print(table)

In [ ]:
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()

In [ ]:
for m in history.messages:
    print(f"{m.name}: {m.content}")

In [ ]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.functions import kernel_function
from utils.config import get_azure_openai_client
from utils.store import get_data_store
import datetime


sku_list =  ["SKU-A100", "SKU-A102", "SKU-A103", "SKU-A104"]

data_store = get_data_store()

avaiilable_skus = await data_store.query_data("SELECT * FROM c", "sku")
skus_dict = {sku["id"]: sku for sku in avaiilable_skus}

# Check if all SKUs are available
invalid_skus = [sku for sku in sku_list if sku not in skus_dict]
invalid_skus


In [ ]:
results = {}

order = [
    {
      "sku": "SKU-A100",
      "description": "Sport T shirt",
      "size": "M",
      "color": "Red",
      "quantity": 4,
      "unit_price": 10.0
    },
    {
      "sku": "SKU-A102",
      "description": "Hoodie",
      "size": "M",
      "color": "Black",
      "quantity": 20,
      "unit_price": 25.0
    },
    {
      "sku": "SKU-A103",
      "description": "Hoodie",
      "size": "L",
      "color": "Black",
      "quantity": 30,
      "unit_price": 25.0
    },
    {
      "sku": "SKU-A104",
      "description": "Elephant T-shirt",
      "size": "S",
      "color": "White",
      "quantity": 5,
      "unit_price": 100.0
    }
  ]

    
facilities = await data_store.query_data(
      f"SELECT * FROM c", 
      "facility"
  )

console.print(f"Facilities for SKU {sku_id}: {facilities}")

for item in order:
    sku_id = item["sku"]
    quantity = item["quantity"]

    
    
    total_available = 0
    locations = []
    
    for facility in facilities:
        for sku_availability in facility.get("skuAvailability", []):
          if sku_id in sku_availability["sku"]:
              available = sku_availability["availableQuantity"]
              total_available += available
              locations.append({
                  "facility_id": facility["id"],
                  "name": facility.get("name", "Unknown"),
                  "available": available
              })
      
    results[sku_id] = {
        "requested": quantity,
        "available": total_available,
        "is_available": total_available >= quantity,
        "locations": locations
    }

results



In [ ]:
from order.validator_agent import validator_agent

In [ ]:
validator_agent.kernel.__dict__

In [ ]:
# Save the modified agent instructions for future use
def save_agent_instructions():
    # Create a dictionary with agent instructions
    instructions = {
        "validator_agent": validator_agent.instructions,
        "substitution_agent": substitution_agent.instructions,
        "pricing_agent": pricing_agent.instructions,
        "fulfillment_agent": fulfillment_agent.instructions
    }
    
    import json
    import os
    
    # Create directory if it doesn't exist
    os.makedirs("./output", exist_ok=True)
    
    # Save instructions to file
    with open("./output/agent_instructions.json", "w") as f:
        json.dump(instructions, f, indent=2)
    
    print("Agent instructions saved to ./output/agent_instructions.json")

# Uncomment to save instructions
# save_agent_instructions()